# Vorlesungswiederholung

### Lasso

$$\min_{w} { \frac{1}{2n_{\text{samples}}} ||X w - y|| ^ 2 + \alpha ||w||}$$

Die Schwierigkeit besteht darin Alpha so zu optimieren, dass das Modell die zugrunde liegenden Daten mit optimalem "Grad der Generalisierung" und Varianz wiedergibt.
Als Kriterium kommen der resultierende MSE-Wert (mean squared error) oder sogenannte Informationskriterien wie das Akaike information criterion (AIC) oder das Bayes information criterion (BIC) in Frage. 

Im Folgenden Beispiel wird Alpha anhand des MSE-Werts optimiert. Mit definierten Werten von Alpha werden Modelle bestimmt. Das beste Modell wird mit k-facher Cross-Validierung ausgewählt.

### Ridge

Bei der Ridge Regression geht der mit $\alpha$ skalierte Fehlerterm quadratisch in das im Abschnitt "Lasso" vorgestellte Minimierungsproblem ein ($L_2$-Regularisierung):

$$\min_{w} { \frac{1}{2n_{\text{samples}}} ||X w - y|| ^ 2 + \alpha ||w|| ^ 2}$$

Ziel ist es einen Kompromiss zwischen Bias und Varianz zu finden. Mit einem hohen $\alpha$ kann Overfitting vermieden werden. Wird $\alpha$ zu groß gewählt, führt dies zu Underfitting (hoher Bias), da die Modellparameter dazu tendieren zu klein zu werden. 

### Elastic Net

Der Elastic Net Algorithmus führt die Regression mit einer Kombination aus $L_1$- und $L_2$-Regularisierung durch. Elasic Net kann genau wie Ridge angewendet werden, sollten mehrere Merkmale miteinander korreliert sein. Das Minimalisierungsproblem lautet folgendermaßen:

$$\min_{w} { \frac{1}{2n_{\text{samples}}} ||X w - y|| ^ 2 + \alpha \rho ||w|| +
\frac{\alpha(1-\rho)}{2} ||w|| ^ 2}$$

Optimiert werden müssen die Parameter für $\alpha$ und $\rho$, auch $l1_{ratio}$ genannt. $\rho$ liegt zwischen 0 und 1.

### K-fache Cross-Validierung

Um Overfitting zu erkennen gibt es die Strategie der Cross-Validierung. Damit soll eine möglichst hohe Generalisierung sichergestellt werden. Jede Art von Wissen über die Daten kann schon bei der Selektion von sogenannten "Hyperparametern" (externe Konfigurationsvariablen) die Modellierung beeinflussen. Um diesen Effekt zu minimieren wird das Trainings-Set unterteilt und ein Validierungs-Set erzeugt. Mit dem Validierungs-Set erfolgt eine Vorbewertung des Modells und somit der Hyperparameter. Nach der Modellierung wird das Modell wie gehabt mit den Test-Daten überprüft. 

Durch die weitere Unterteilung senkt sich aber wiederum die Anzahl der verfügbaren Stichproben und das Modell könnte an Aussagekraft verlieren. Die Auswahl der Daten hätte einen zufälligen Einfluss auf das Modell. 

Bei der k-fachen Cross-Validierung wird dieses Problem umgangen, indem die Trainingsdaten in k kleinere Datensätze unterteilt werden. Anschließend werden k Schritte durchlaufen, welche immer den gleichen Ablauf haben: Ein Modell wird anhand von k-1 Trainingdatensätzen erstellt und mit dem übrigen Datensatz validiert (seine Performance bestimmt). Der Durchschnitt aller k Validierungen ergibt die Gesamtperformance. 

<img src="k-fache_Cross_Validierung.png" width="550">

# Interaktiver Teil
**Ziel**: Erstellung eines Modells zur Vorhersage der Zugfestigkeit

<img src="TrainingValidationPlan.png" width="450"> 

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv(r"Luo_Q_2023.1_Vereinfacht.csv", sep=';', decimal='.')
data[:10]

Wir sehen uns die Spalten der Daten einmal vollständig an und fassen sie in die Unterschiedlichen Kategorien zusammen. 

In [ ]:
data.keys()

In [ ]:
manufacturing_parameter = ['Power', 'Speed']

mechanical_parameter = ['Strength_Mean', 'Strength_Std', 'Yield_mean', 'Yield_Std', 'Uniform_Mean', 'Uniform_Std',
                        'Ductility_Mean', 'Ductility_Std', 'Youngs_Modulus_Mean', 'Youngs_Modulus_Std']

std_parameter = ['Avizo_Porosity_Std', 'Archimedes_porosity_Std','Diameter_Avizo_Std', 'Sphericity_Avizo_Std', 
                 'Roughness_Avg_Std', 'Roughness_RMS_Std', 'Grain_diameter_Std', 'Grain_aspect_Std', 'Vicker_Std',
                'Strength_Std', 'Yield_Std', 'Uniform_Std', 'Ductility_Std', 'Youngs_Modulus_Std']

pore_parameter = ['Avizo_Porosity_Mean', 'Archimedes_porosity_Mean', 'Archimedes_porosity_Std',
                  'Pore_quantity_Sum', 'Diameter_Avizo_Mean', 'Diameter_Avizo_Std', 'Projected_Pore_area_Avg', 
                  'Projected_Pore_area_Max', 'Projected_Pore_area_Sum', 'Sphericity_Avizo_Mean', 'Sphericity_Avizo_Std', 
                  'Roughness_Avg_Mean', 'Roughness_Avg_Std', 'Roughness_RMS_Mean', 'Roughness_RMS_Std']

In [ ]:
#Separate train and test sets
x = data.drop(labels=(manufacturing_parameter+mechanical_parameter+std_parameter), axis=1)
y = data['Strength_Mean']

x[:5]

Hier werden unterschiedliche Zusammenhänge geplottet. Bei einigen Parameter sind klare lineare Zusammenhänge zu erkennen.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x = data['Strength_Mean'], y = data['Archimedes_porosity_Std']) #% Change Data

Nicht alle Werte wurden ermittelt. Um diese dennoch zu verwenden, werden diese auf den Mittelwert gesetzt. Bei der Verwendung einer Regularisierung wie Lasso ($L_1$), Ridge ($L_2$) oder beide gleichzeitig ElasticNet müssen die Daten normalisiert werden. Die hier durchgeführte Datenoperation skaliert die Daten so, dass sie einen Mittelwert von 0 und Standardabweichung von 1 erreichen.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Handling missing values by filling them with the mean of each column
x_without_NaN = x.fillna(x.mean()) 

scaler = StandardScaler()
scaler.fit(x_without_NaN)
x_norm = scaler.transform(x_without_NaN)

print(x_norm.mean(axis=0))
print(x_norm.std(axis=0))

In [ ]:
y_np = y.to_numpy() #Convert to numpy
print(y_np.shape)
y_np = y_np[:,np.newaxis]
print(y_np.shape)

Es wird eine Anpasung eines Modelles mit Hilfe einer Kreuzvalidierung durchgeführt. Hierfür stehen unterschiedliche Modelle zur Verfügung.

____
## Modellbildung

Parameter für Lasso, alpha ist die $L_1$-Regularisierung

```
sklearn.linear_model.Lasso(alpha=1.0, *, fit_intercept=True, precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
```

---
Parameter für Ridge, alpha ist die $L_2$-Regularisierung
```
sklearn.linear_model.Ridge(alpha=1.0, *, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None)
```

---
Parameter für Elastic Net, alpha ist die Gesamtregulierung aus $l1_{ratio} * L_1 + L_2$
```
sklearn.linear_model.ElasticNet(alpha=1.0, *, l1_ratio=0.5, fit_intercept=True, precompute=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
```

Das Modell wird gefittet. Zu diesem Zweck wird eine 5-Fold-Kreuzvalidierung durchgeführt. Der Einfluss des Modells auf die Vorhersagegenauigkeit und die Varianz zwischen den Falten (Folds) wird ausgegeben.

Zur Variation der Modelle können die verschiedenen Modelle mit dem Zeichen **#** ein- bzw. auskommentiert werden. Zusätzlich kann die Regularisierungsstärke über den **alpha**-Wert eingestellt werden.

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE

kf = KFold(n_splits=5)

scores = []
models = []

plt.figure()
for fold, (idx_train, idx_test) in enumerate(kf.split(x_norm)):
    x_train = x_norm[idx_train]
    y_train = y_np[idx_train]
    x_test = x_norm[idx_test]
    y_test = y_np[idx_test]
    
    model = LinearRegression() #% Change model
    # model = Lasso(alpha=1.0)
    # model = Ridge(alpha=0.1)
    # model = ElasticNet(alpha=1.0, l1_ratio=0.5)
    model.fit(x_train,y_train)
    
    y_predict = model.predict(x_test)
    score = MSE(y_test, y_predict)
    plt.scatter(x = y_test, y = y_predict, label = str(fold))
    
    scores.append(score)
    models.append(model)

plt.legend()
plt.xlabel("Real value")
plt.ylabel("Predicted value")

print(f'Scores of each fold {scores}')
print(f'Mean score: {np.mean(scores)}')
print(f'Standard deviation: {np.std(scores)}')

In den folgenden Zellen werden die Gewichte des angepassten Modells betrachtet. Sie werden für jeden Fold (Faltung) getrennt ausgewertet und dargestellt. Durch die 5-fache Kreuzvalidierung liegen somit 5 verschiedene Variationen von Trainings- und Testdaten vor, die jeweils zu 5 verschiedenen Modellen und dementsprechend Gewichten führen.

In [ ]:
keys = x_without_NaN.keys()

plt.figure(figsize=(8,3))
for fold, model in enumerate(models):
    plt.plot(np.squeeze(model.coef_), label = str(fold))
plt.xticks(range(len(keys)), keys, rotation='vertical')
plt.legend()
plt.show()

**Aufgabe: Wie verhalten sich die unterschiedlichen Modelle (LinearRegression, Lasso, Ridge) bei Veränderung der Werte für die Regularisierung (alpha)?**

Untersuche die Modelle hinsichtlich:
- Anpassungsgüte
- Varianz zwischen den Folds
- Auswirkungen auf die Gewichte